In [1]:
import pandas as pd
import numpy as np

import keras as K
import tensorflow as tf
import time

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
class Config(object):
    pass

config = Config()
config.log_dir = 'tf_logs/nn/'+str(time.strftime('%Y-%m%d %H:%M:%S'))
config.batch_size = 400
config.epochs = 400

In [3]:
# fix random seed for reproducibility
np.random.seed(42)

In [4]:
from dance.audiosamples import RectifiedAudioSamples as AudioSamples

In [5]:
train_data = pd.read_pickle("train_data.pkl")
val_data = pd.read_pickle("val_data.pkl")
test_data = pd.read_pickle("test_data.pkl")

In [6]:
def generate_samples(train_data, val_data, test_data, sampler):
    try:
        del(train_samples)
        del(val_samples)
        del(test_samples)
    except:
        pass
    train_samples = sampler(train_data, subsample_rate=2)
    scaler = train_samples.norm()
    encoder = train_samples.encode()
    val_samples=sampler(val_data, subsample_rate=2)
    val_samples.norm(scaler)
    val_samples.encode(encoder)
    test_samples = sampler(test_data, subsample_rate=2)
    test_samples.norm(scaler)
    test_samples.encode(encoder)
    return train_samples, val_samples, test_samples

In [7]:
train, val, test = generate_samples(train_data, val_data, test_data, AudioSamples)

In [8]:
tf.reset_default_graph()
model = K.models.Sequential()
model.add(K.layers.Dense(500, activation='softplus', input_dim=train.X.shape[1], kernel_regularizer=K.regularizers.l2()))
# model.add(K.layers.Dense(250, activation='softplus', kernel_regularizer=K.regularizers.l2()))
model.add(K.layers.Dense(100, activation='softplus', kernel_regularizer=K.regularizers.l2()))
# model.add(K.layers.Dense(50, activation='softplus', kernel_regularizer=K.regularizers.l2()))
model.add(K.layers.Dense(train.Y.shape[-1], activation='softmax', kernel_regularizer=K.regularizers.l2()))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [9]:
tbCallback = K.callbacks.TensorBoard(config.log_dir, batch_size=config.batch_size, histogram_freq=10)
patience = K.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

history = model.fit(train.X, train.Y, epochs=config.epochs, 
                    batch_size=config.batch_size, 
                    validation_data=(val.X, val.Y), 
                    callbacks=[tbCallback, patience],
                    verbose=True,
                   )

Train on 39648 samples, validate on 10166 samples
INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.
INFO:tensorflow:Summary name dense_1/bias:0 is illegal; using dense_1/bias_0 instead.
INFO:tensorflow:Summary name dense_2/kernel:0 is illegal; using dense_2/kernel_0 instead.
INFO:tensorflow:Summary name dense_2/bias:0 is illegal; using dense_2/bias_0 instead.
INFO:tensorflow:Summary name dense_3/kernel:0 is illegal; using dense_3/kernel_0 instead.
INFO:tensorflow:Summary name dense_3/bias:0 is illegal; using dense_3/bias_0 instead.
Epoch 1/400
39648/39648 [==============================] - 18s - loss: 8.2025 - acc: 0.3219 - val_loss: 4.6061 - val_acc: 0.2746
Epoch 2/400
39648/39648 [==============================] - 15s - loss: 3.1613 - acc: 0.3961 - val_loss: 2.9639 - val_acc: 0.2787
Epoch 3/400
39648/39648 [==============================] - 15s - loss: 2.4162 - acc: 0.4018 - val_loss: 2.6826 - val_acc: 0.1844
Epoch 4/400
39648/39648 [==========

In [10]:
model.evaluate(test.X, y=test.Y, verbose=True)

18240/18384 [============================>.] - ETA: 0s

[2.3938857830535651, 0.22193211488250653]